In [1]:
import pandas as pd
import numpy as np
import nltk
import contractions
import re
import gensim
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec
import gensim.downloader
from gensim.test.utils import datapath
from gensim import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import accuracy_score

#### Importing the dataset

In [2]:
df= pd.read_table('amazon_reviews_us_Beauty_v1_00.tsv',on_bad_lines='skip',low_memory=False)

In [3]:
df.head(10)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,1797882,R3I2DHQBR577SS,B001ANOOOE,2102612,The Naked Bee Vitmin C Moisturizing Sunscreen ...,Beauty,5,0.0,0.0,N,Y,Five Stars,"Love this, excellent sun block!!",2015-08-31
1,US,18381298,R1QNE9NQFJC2Y4,B0016J22EQ,106393691,"Alba Botanica Sunless Tanning Lotion, 4 Ounce",Beauty,5,0.0,0.0,N,Y,Thank you Alba Bontanica!,The great thing about this cream is that it do...,2015-08-31
2,US,19242472,R3LIDG2Q4LJBAO,B00HU6UQAG,375449471,"Elysee Infusion Skin Therapy Elixir, 2oz.",Beauty,5,0.0,0.0,N,Y,Five Stars,"Great Product, I'm 65 years old and this is al...",2015-08-31
3,US,19551372,R3KSZHPAEVPEAL,B002HWS7RM,255651889,"Diane D722 Color, Perm And Conditioner Process...",Beauty,5,0.0,0.0,N,Y,GOOD DEAL!,I use them as shower caps & conditioning caps....,2015-08-31
4,US,14802407,RAI2OIG50KZ43,B00SM99KWU,116158747,Biore UV Aqua Rich Watery Essence SPF50+/PA+++...,Beauty,5,0.0,0.0,N,Y,this soaks in quick and provides a nice base f...,This is my go-to daily sunblock. It leaves no ...,2015-08-31
5,US,2909389,R1R30FA4RB5P54,B000NYL1Z6,166146615,Murad Clarifying Cleanser,Beauty,4,0.0,0.0,N,Y,Four Stars,Good,2015-08-31
6,US,19397215,R30IJKCGJBGPJH,B001SYWTFG,111742328,CoverGirl Queen Collection Perfect Point Plus ...,Beauty,5,0.0,0.0,N,Y,Good buy,"Great eyeliner, does run just a little bit at ...",2015-08-31
7,US,3195210,R18GLJJPVQ1OVH,B005F2EVMQ,255803087,"Bifesta Mandom Eye Makeup Remover, 145ml",Beauty,5,0.0,0.0,N,Y,Five Stars,Best makeup remover！,2015-08-31
8,US,52216383,R8TVYIJXLYJT0,B00M1SUW7K,246816549,Can You Handlebar All-Natural Secondary Mousta...,Beauty,5,0.0,0.0,N,Y,Tame the wild mustache,This is a great product. I'm on my latest stab...,2015-08-31
9,US,10278216,R1CJGF6M3PVHEZ,B001KYQA1S,9612905,"Maybelline Great Lash Washable Mascara, Clear ...",Beauty,1,0.0,2.0,N,Y,but it's like having nothing on them at all,I thought it would darken the lashes even thou...,2015-08-31


In [4]:
df_fnl=df[['review_body','star_rating']]

#### Replacing the ratings and creating three classes

In [5]:
df_final=df_fnl.replace({'star_rating':{2:1,3:2,4:3,5:3,'5':3,'2':1,'3':2,'4':3,'1':1}})


In [6]:
df_final.isnull().sum()

review_body    400
star_rating     10
dtype: int64

In [7]:
df_final.dropna(inplace=True)

In [8]:
s0 = df_final[df_final['star_rating'].eq(1)].sample(20000).index
s1 = df_final[df_final['star_rating'].eq(2)].sample(20000).index 
s2 = df_final[df_final['star_rating'].eq(3)].sample(20000).index 

df_fi = df_final.loc[s0.union(s1).union(s2)]

In [9]:
df_fi['star_rating'].value_counts()

2    20000
3    20000
1    20000
Name: star_rating, dtype: int64

#### Applying the datacleaning techniques to lower, remove white spaces between words, contractions and HTMl and URL Tags

In [10]:
def remove_alphanumeric(s):
    s=s.lower()
    s=s.strip()
    s=contractions.fix(s)
    s=s.replace(r'<[^<>]*>', '')
    s=s.replace(r'http\S+', '').replace(r'www\S+', '')
    s=re.sub(r'[^a-zA-Z]',' ',s)
    return re.sub(' +', ' ',s)

In [11]:
df_fi['review_body']=df_fi['review_body'].apply(remove_alphanumeric)

In [12]:
df_fi

,review_body,star_rating
106,no complaints from my wife and buying from ama...,2
231,i was super excited about whitening with this ...,2
510,good size for travel and good price but they c...,3
594,i want my money i did not respond nothing,1
696,i love this brush br i now own of them,3
...,...,...
5093960,this is my only experience with an ionic hair ...,2
5094055,for years crest was the toothpaste of choice i...,3
5094209,after reading all the reviews i was alittle wa...,2
5094267,i bought this for my wife as a christmas prese...,1


### Word Embedding 

Part a

In [13]:
google_wrd2vc = gensim.downloader.load('word2vec-google-news-300')

In [14]:
print(google_wrd2vc.most_similar(positive=['woman', 'king'], negative=['man']))

[('queen', 0.7118193507194519), ('monarch', 0.6189674735069275), ('princess', 0.5902431011199951), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321243286133), ('kings', 0.5236844420433044), ('Queen_Consort', 0.5235945582389832), ('queens', 0.518113374710083), ('sultan', 0.5098593831062317), ('monarchy', 0.5087411999702454)]


In [15]:
google_wrd2vc.similarity('good','better')

0.61207294

In [16]:
google_wrd2vc.similarity('machine','sounds')

0.16873193

In [17]:
print(google_wrd2vc.most_similar(positive=['king', 'woman'], negative=['man'], topn=1))

[('queen', 0.7118193507194519)]


Part b

In [18]:

class MyCorpus:
    ## step to create custom Corpus for Word2vec to use to generate word vectors
    def __iter__(self):
        for line in df_fi['review_body']:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [19]:
#Loading my own Word2Vec 
my_model= Word2Vec(sentences = MyCorpus(), vector_size = 300, window = 13, min_count = 9)

In [20]:
my_model.wv.similarity('machine','sounds')

0.15689024

In [21]:
my_model.wv.similarity('good','better')

0.38715005

In [22]:
print(my_model.wv.most_similar(positive=['king', 'woman'], negative=['man'],topn=1))

[('concealing', 0.6018328070640564)]


The performance of the Pre-trained model by word2vec-google-news-300 is better since it has a much bigger corpus than my personal dataset. Therefore the pretrained mmodel can findout better similarities than my model.

#### Simple models

In [23]:
#Splitting data into train and test dataset

true_label = df_fi['star_rating'].to_numpy()
X_train, X_test, Y_train, Y_test = train_test_split(df_fi['review_body'], true_label, test_size=0.2, random_state=42)

In [24]:
##average Word2Vec vectors for each review as the input feature 

for row in df_fi['review_body']:
    sent_split=row.split(' ')
    for word in sent_split:
        if word in google_wrd2vc:
            
        
        
    

IndentationError: expected an indented block (1370460310.py, line 10)